# Fake News Detection

## Inspiration
With every news channel constantly wanting to attract more and more consumers on their page, it becomes increasingly hard to determine which news articles are real and which are seeking clicks through false headlines and false information. I personally do not want to be misled into believing something that is not true so I wanted to create an app that would tell me whether certain news is true or not.

We see the consequences of fake news in our culture everyday. A good example would be fake news about celebrities that could ruin their image.

In [1]:
# import necessary libraries to work with data and nlp models

import tensorflow
import torch
from transformers import pipeline
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
# read in the data
all_data_news = pd.read_csv('WELFake_Dataset.csv', index_col=0)
# label: 0 = fake, 1 = real
all_data_news.head(2)

,title,text,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,NaN,Did they post their votes for Hillary already?,1


## Plan
We will be using Hugging Face, an AI company, that offers pretrained models (which includes NLP models), to detect fake news.

**Why not train our own model?** An NLP model requires extensive data which often takes days to train, and takes up a lot of space. It is also unethical to train your own data when there are pretrained models which already exist.

#### Split Data

In [3]:
X = all_data_news[['title', 'text']]
y = all_data_news[['label']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

We'll first use the fake news model given to us from Hugging Face. Then we'll fine tune our own fake news model.

In [4]:
model = 'jy46604790/Fake-News-Bert-Detect'
classifier = pipeline('sentiment-analysis', model=model)

In [5]:
news_titles = X_test['title']
news_text = X_test['text']
print('Number of NaN titles:', news_titles.isna().sum())
print('Number of NaN texts:', news_text.isna().sum())
print('Rows w/ NaN title or NaN text:', X_test.isna().any(axis=1).sum())
print('Rows w/ NaN title and NaN text:', (X_test.isna().sum(axis=1) > 1).sum())

Number of NaN titles: 105
Number of NaN texts: 7
Rows w/ NaN title or NaN text: 112
Rows w/ NaN title and NaN text: 0


We see from above that for every row that has a NaN title, it was a text value that is NOT NaN, so therefore we can replace all NaN title's with their corresponding text value to check for fake news.

In [6]:
X_test['title'].fillna(X_test['text'], inplace=True)
filled_news_titles = X_test['title']
print('New Number of NaN titles:', filled_news_titles.isna().sum())

New Number of NaN titles: 0


In [7]:
filled_news_titles = list(filled_news_titles)

In [8]:
# this is how i was able to move the predictions from the Hugging Face model to a csv file

# tokenizer_kwargs = {'truncation':True,'max_length':512}
# class_dict = classifier(filled_news_titles,**tokenizer_kwargs)
# scores = pd.DataFrame({'real':class_dict})
# scores.to_csv('scores.csv')


In [9]:
scores = pd.read_csv('scores.csv', index_col=0)
scores.head(2)

,real
0,"{'label': 'LABEL_0', 'score': 0.9990565180778503}"
1,"{'label': 'LABEL_0', 'score': 0.9979349374771118}"


In [10]:
scores.loc[scores['real'].str.contains('LABEL_0'), 'real'] = '0'
scores.loc[scores['real'].str.contains('LABEL_1'), 'real'] = '1'
scores.head(2)

,real
0,0
1,0


In [11]:
from sklearn.metrics import accuracy_score

scores['real'] = pd.to_numeric(scores['real'])
pred = scores['real']
real = y_test['label']
print('\'jy46604790/Fake-News-Bert-Detect\' Accuracy Score:', accuracy_score(real, pred))

'jy46604790/Fake-News-Bert-Detect' Accuracy Score: 0.24911624038261593
